In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
from random import randrange
from datetime import timedelta
import numpy as np

def random_date(start, end):
    delta = (end - start).days
    if delta==0:
        return start
    random_day = randrange(delta)
    return start + timedelta(days=random_day)

In [2]:
aisles = pd.read_csv("flat_sources/aisles.csv")
departments = pd.read_csv("flat_sources/departments.csv")
order_products_prior = pd.read_csv("flat_sources/order_products__prior.csv")
order_products_train = pd.read_csv("flat_sources/order_products__train.csv")
orders = pd.read_csv("flat_sources/orders.csv")
products = pd.read_csv("flat_sources/products.csv")
uscities = pd.read_csv("flat_sources/uscities.csv")
WeatherEvents = pd.read_csv("flat_sources/WeatherEvents_Jan2016-Dec2021.csv")
orders['days_since_prior_order'] = orders['days_since_prior_order'].fillna(0)

In [3]:
sum_days_since_prior = orders[['user_id', 'days_since_prior_order']].groupby(['user_id']).sum()
day_of_last_order = date(2017, 1, 1) - timedelta(days=1.0)
first_day = date(2016, 1, 1)
sum_days_since_prior['first_order_date'] = sum_days_since_prior['days_since_prior_order'].apply(lambda x: day_of_last_order - timedelta(days=x))
sum_days_since_prior['first_order_date'] = sum_days_since_prior['first_order_date'].apply(lambda x: random_date(first_day, x))
sum_days_since_prior = sum_days_since_prior.reset_index()
ddd = WeatherEvents['City'].unique()
cities_singular = uscities['city'].value_counts()[uscities['city'].value_counts() == 1].index
overlaping_datasets_cities=ddd[np.isin(ddd, cities_singular)]
sum_days_since_prior['user_location'] = np.random.choice(overlaping_datasets_cities, size=len(sum_days_since_prior))
orders['days_since_first_order']=orders.groupby(['user_id'])['days_since_prior_order'].cumsum()

In [4]:
sum_days_since_prior

,user_id,days_since_prior_order,first_order_date,user_location
0,1,190.0,2016-04-23,Angleton
1,2,228.0,2016-03-10,Erlanger
2,3,144.0,2016-05-09,West Palm Beach
3,4,85.0,2016-01-07,Rancho Cordova
4,5,46.0,2016-02-21,Cloquet
...,...,...,...,...
206204,206205,50.0,2016-08-29,Kaycee
206205,206206,249.0,2016-02-07,Fort Stockton
206206,206207,229.0,2016-01-04,North Clarendon
206207,206208,361.0,2016-01-04,New Roads


In [5]:
orders = pd.merge(orders, sum_days_since_prior[['user_id', 'first_order_date', 'user_location']], on='user_id')
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,days_since_first_order,first_order_date,user_location
0,2539329,1,prior,1,2,8,0.0,0.0,2016-04-23,Angleton
1,2398795,1,prior,2,3,7,15.0,15.0,2016-04-23,Angleton
2,473747,1,prior,3,3,12,21.0,36.0,2016-04-23,Angleton
3,2254736,1,prior,4,4,7,29.0,65.0,2016-04-23,Angleton
4,431534,1,prior,5,4,15,28.0,93.0,2016-04-23,Angleton
...,...,...,...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0,155.0,2016-03-11,Jordan Valley
3421079,1854736,206209,prior,11,4,10,30.0,185.0,2016-03-11,Jordan Valley
3421080,626363,206209,prior,12,1,12,18.0,203.0,2016-03-11,Jordan Valley
3421081,2977660,206209,prior,13,1,12,7.0,210.0,2016-03-11,Jordan Valley


In [6]:
cities_singular = uscities['city'].value_counts()[uscities['city'].value_counts() == 1].index

In [7]:
orders['order_date'] = orders.apply(lambda x: x['first_order_date'] + timedelta(days=x['days_since_first_order']), axis=1)

In [9]:
orders.to_csv('flat_sources/orders2.csv')

In [2]:
WeatherEvents = pd.read_csv("flat_sources/WeatherEvents_Jan2016-Dec2021.csv")
print(WeatherEvents['City'].value_counts())

Jacksonville          31112
Columbus              30351
Cleveland             28164
Washington            26587
Greenville            26121
                      ...  
Commerce                 54
Campo                    43
Hanksville               34
Colville                 27
South Padre Island        8
Name: City, Length: 1716, dtype: int64


In [17]:
WeatherEvents['StartDate'] = WeatherEvents['StartTime(UTC)'].apply(lambda x: x[:10])
WeatherEvents['StartYear'] = WeatherEvents['StartTime(UTC)'].apply(lambda x: x[:4])

In [18]:
print(WeatherEvents[WeatherEvents['StartYear']=='2016'][['City',  'StartDate']].value_counts())

City             StartDate 
Cody             2016-09-24    102
Jacksonville     2016-10-07     93
Tampa            2016-08-31     91
Chesapeake       2016-09-21     90
Tampa            2016-06-07     82
                              ... 
North Dinwiddie  2016-07-23      1
Corona           2016-06-05      1
North Dinwiddie  2016-07-21      1
Corona           2016-06-08      1
Zionsville       2016-12-31      1
Length: 294157, dtype: int64


In [61]:
W2016eather = WeatherEvents[WeatherEvents['StartYear']=='2016'].drop_duplicates(subset=['StartDate', 'City'], keep='first').dropna(subset=['City'])
W2016eather = W2016eather.drop(['StartDate', 'StartYear'], axis=1)
W2016eather.to_csv('flat_sources/weather2016.csv')